This notebook has been used for data cleaning of the data sources and in the end I store the data into a new csv file.

In [1]:
import pandas as pd
import numpy as np
import geopandas

/Users/antonis1991/opt/anaconda3/lib/python3.8/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
<ipython-input-1-b8374a4f173f>:3: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migratio

### Reading the files

In [2]:
# read the files and store them into a list
import glob
csv_files = glob.glob('*.csv')


# identify the names of the cities 
cities = []
for file in csv_files:
    cities.append(file[:-4])

# read the firs csv file and include a column with the city name
data = pd.read_csv(csv_files[0])
where = [cities[0] for i in range(data.shape[0])]
data['City'] = where

# continue with the rest cities and append the csv file to the initial data
for i, file in enumerate(csv_files[1:]):
    data_file = pd.read_csv(file)
    where = [cities[i+1] for j in range(data_file.shape[0])]
    data_file['City'] = where
    data = data.append(data_file, ignore_index=True)

In [3]:
data

,Unnamed: 0,realSum,room_type,room_shared,room_private,person_capacity,host_is_superhost,multi,biz,cleanliness_rating,...,bedrooms,dist,metro_dist,attr_index,attr_index_norm,rest_index,rest_index_norm,lng,lat,City
0,0,194.033698,Private room,False,True,2.0,False,1,0,10.0,...,1,5.022964,2.539380,78.690379,4.166708,98.253896,6.846473,4.90569,52.41772,Amsterdam
1,1,344.245776,Private room,False,True,4.0,False,0,0,8.0,...,1,0.488389,0.239404,631.176378,33.421209,837.280757,58.342928,4.90005,52.37432,Amsterdam
2,2,264.101422,Private room,False,True,2.0,False,0,1,9.0,...,1,5.748312,3.651621,75.275877,3.985908,95.386955,6.646700,4.97512,52.36103,Amsterdam
3,3,433.529398,Private room,False,True,4.0,False,0,1,9.0,...,2,0.384862,0.439876,493.272534,26.119108,875.033098,60.973565,4.89417,52.37663,Amsterdam
4,4,485.552926,Private room,False,True,2.0,True,0,0,10.0,...,1,0.544738,0.318693,552.830324,29.272733,815.305740,56.811677,4.90051,52.37508,Amsterdam
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17929,4609,310.449446,Private room,False,True,2.0,False,0,0,10.0,...,1,6.347654,0.445862,194.410695,13.513703,446.486289,7.994710,-0.21207,51.48667,London
17930,4610,265.057974,Entire home/apt,False,False,4.0,False,1,0,8.0,...,1,5.231706,0.463949,254.476513,17.688945,537.720506,9.628334,-0.05459,51.52018,London
17931,4611,142.289329,Private room,False,True,2.0,False,0,0,10.0,...,1,8.867238,2.675007,125.891017,8.750824,266.789887,4.777096,-0.12056,51.42875,London
17932,4612,372.304146,Private room,False,True,2.0,False,0,0,8.0,...,1,7.576798,1.682697,146.161215,10.159828,325.152018,5.822118,-0.12810,51.44023,London


# Data Cleaning

In [4]:
data.isna().sum()

Unnamed: 0                    0
realSum                       0
room_type                     0
room_shared                   0
room_private                  0
person_capacity               0
host_is_superhost             0
multi                         0
biz                           0
cleanliness_rating            0
guest_satisfaction_overall    0
bedrooms                      0
dist                          0
metro_dist                    0
attr_index                    0
attr_index_norm               0
rest_index                    0
rest_index_norm               0
lng                           0
lat                           0
City                          0
dtype: int64

### Check duplicated values

In [5]:
data.duplicated().sum()

0

### Drop off some columns

I will drop off the columns that I won't use.

In [6]:
data = data.drop(["Unnamed: 0","room_shared", "room_private","attr_index",
                  "attr_index_norm","rest_index","rest_index_norm","bedrooms",
                  "host_is_superhost","multi","biz","cleanliness_rating","guest_satisfaction_overall",
                  "dist","metro_dist"], axis=1)

### Rename some columns

In [7]:
data = data.rename({'realSum':'Price', 'room_type':'Room type', 'person_capacity':'Person capacity',
                    'lng':'Longitude', 'lat':'Latitude'}, axis=1)

In [8]:
data['Price'] = np.around(data.Price.values,2)
data

,Price,Room type,Person capacity,Longitude,Latitude,City
0,194.03,Private room,2.0,4.90569,52.41772,Amsterdam
1,344.25,Private room,4.0,4.90005,52.37432,Amsterdam
2,264.10,Private room,2.0,4.97512,52.36103,Amsterdam
3,433.53,Private room,4.0,4.89417,52.37663,Amsterdam
4,485.55,Private room,2.0,4.90051,52.37508,Amsterdam
...,...,...,...,...,...,...
17929,310.45,Private room,2.0,-0.21207,51.48667,London
17930,265.06,Entire home/apt,4.0,-0.05459,51.52018,London
17931,142.29,Private room,2.0,-0.12056,51.42875,London
17932,372.30,Private room,2.0,-0.12810,51.44023,London


### Remove some rows

I want to check the available options of the Room type.

In [9]:
for city in cities:
    print('-'*60)
    print(city)
    print()
    print(round((data[data.City==city]['Room type'].value_counts()/data[data.City==city]['Room type'].value_counts().sum())*100,2))
    print('-'*60)
    print('-'*60)

------------------------------------------------------------
Amsterdam

Private room       50.68
Entire home/apt    48.78
Shared room         0.54
Name: Room type, dtype: float64
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
Vienna

Entire home/apt    77.62
Private room       21.98
Shared room         0.40
Name: Room type, dtype: float64
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
Rome

Entire home/apt    61.31
Private room       38.54
Shared room         0.16
Name: Room type, dtype: float64
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
Lisbon

Entire home/apt    67.48
Private roo

The option of shared room has a very small percentage. Thus, I drop it off.

In [10]:
data = data.drop(data[data['Room type']=="Shared room"].index, axis=0)
data.reset_index(inplace=True)
data = data.drop("index",axis=1)

In [11]:
data

,Price,Room type,Person capacity,Longitude,Latitude,City
0,194.03,Private room,2.0,4.90569,52.41772,Amsterdam
1,344.25,Private room,4.0,4.90005,52.37432,Amsterdam
2,264.10,Private room,2.0,4.97512,52.36103,Amsterdam
3,433.53,Private room,4.0,4.89417,52.37663,Amsterdam
4,485.55,Private room,2.0,4.90051,52.37508,Amsterdam
...,...,...,...,...,...,...
17804,310.45,Private room,2.0,-0.21207,51.48667,London
17805,265.06,Entire home/apt,4.0,-0.05459,51.52018,London
17806,142.29,Private room,2.0,-0.12056,51.42875,London
17807,372.30,Private room,2.0,-0.12810,51.44023,London


# Neighbourhoods

In [12]:
nbh_geo_amsterdam = geopandas.read_file('Amsterdam_neighbourhoods/neighbourhoods.geojson', driver='GeoJSON')
nbh_geo_lisbon = geopandas.read_file('Lisbon_neighbourhoods/neighbourhoods.geojson', driver='GeoJSON')
nbh_geo_london = geopandas.read_file('London_neighbourhoods/neighbourhoods.geojson', driver='GeoJSON')
nbh_geo_paris = geopandas.read_file('Paris_neighbourhoods/neighbourhoods.geojson', driver='GeoJSON')
nbh_geo_rome = geopandas.read_file('Rome_neighbourhoods/neighbourhoods.geojson', driver='GeoJSON')
nbh_geo_vienna = geopandas.read_file('Vienna_neighbourhoods/neighbourhoods.geojson', driver='GeoJSON')

nbh_geo_files = {'Amsterdam': nbh_geo_amsterdam,
                 'Lisbon': nbh_geo_lisbon,
                 'London': nbh_geo_london,
                 'Paris': nbh_geo_paris,
                 'Rome': nbh_geo_rome,
                 'Vienna': nbh_geo_vienna}

In [13]:
from shapely.geometry import Point

neighbourhoods = []
for city in cities:
    
    geo_file = nbh_geo_files[city]
    
    dataset=data[data.City==city]
    dataset = dataset.reset_index()
    city_length = dataset.shape[0]
    boroughs = [''] * city_length
    
    for i in range(city_length):
        j=0
        while j<len(geo_file.geometry.values):   
            specific_point = Point(dataset["Longitude"][i],dataset["Latitude"][i])
            neighbourhood = geo_file.geometry.values[j]
            if specific_point.within(neighbourhood):
                boroughs[i] = geo_file.neighbourhood[j]
                break
            j+=1
    neighbourhoods.append(boroughs)

Include the Neighbourhoods in the dataset:

In [14]:
data["Neighbourhood"] = [borough for sublist in neighbourhoods for borough in sublist]

Let's see how many Neighbourhoods are included per city

In [15]:
for city in cities:
    print('-'*60)
    print(city)
    print()
    print(data[data.City==city].Neighbourhood.value_counts())
    print('-'*60)
    print('-'*60)

------------------------------------------------------------
Amsterdam

Centrum-West                              207
Centrum-Oost                              126
De Pijp - Rivierenbuurt                   116
De Baarsjes - Oud-West                    105
Zuid                                       85
Westerpark                                 54
Bos en Lommer                              47
Oostelijk Havengebied - Indische Buurt     47
Oud-Oost                                   40
Noord-West                                 32
Oud-Noord                                  30
Watergraafsmeer                            29
IJburg - Zeeburgereiland                   27
Gaasperdam - Driemond                      23
Slotervaart                                20
De Aker - Nieuw Sloten                     19
Geuzenveld - Slotermeer                    18
Osdorp                                     17
Noord-Oost                                 16
Bijlmer-Centrum                            16
Buitenve

There are cases in some cities (Amsterdam, Athens, ...) which do not belong to any neighbourhood. Let's find them:

In [16]:
index_no_neighbourhood = data[data["Neighbourhood"]==""].index

data.iloc[index_no_neighbourhood,:]

,Price,Room type,Person capacity,Longitude,Latitude,City,Neighbourhood
8,1001.80,Entire home/apt,4.0,4.86459,52.40175,Amsterdam,
787,351.04,Entire home/apt,2.0,4.83378,52.38646,Amsterdam,
799,639.98,Entire home/apt,4.0,4.83529,52.38693,Amsterdam,
800,297.61,Entire home/apt,2.0,4.83382,52.38652,Amsterdam,
13253,466.38,Entire home/apt,3.0,-0.14869,51.48421,London,
13609,505.19,Entire home/apt,4.0,-0.18262,51.46766,London,
13706,202.26,Private room,2.0,-0.17600,51.47600,London,
13903,116.42,Private room,2.0,0.04385,51.49536,London,
13952,174.75,Private room,2.0,-0.05526,51.50148,London,
13984,138.53,Private room,2.0,-0.00662,51.50389,London,


Delete those rows

In [17]:
data = data.drop(index_no_neighbourhood)
data.reset_index(inplace=True)
data = data.drop("index",axis=1)

### Rename the option Entire home/apt -> Entire home

In [18]:
for i in range(data.shape[0]):
    if data.iloc[i,1]=='Entire home/apt':
        data.iloc[i,1]='Entire home'

### Correct the name of some Neighbourhoods

The Neighbourhoods which include letters with Umlaut, like **ü - ö -...**, and § (which is coming from ß->ss) have to be corrected. These cases are in Vienna. Let's give them an eye:

In [19]:
data[data.City=='Vienna'].Neighbourhood.value_counts()

Leopoldstadt            189
Landstra§e              132
Rudolfsheim-Fnfhaus    130
Favoriten               107
Innere Stadt            104
Ottakring                98
Margareten               96
Alsergrund               91
Wieden                   87
Mariahilf                85
Neubau                   84
Meidling                 73
Donaustadt               58
Hernals                  56
Penzing                  55
Brigittenau              53
Josefstadt               52
Whring                  51
Dbling                  42
Simmering                26
Hietzing                 25
Floridsdorf              22
Liesing                  15
Name: Neighbourhood, dtype: int64

In [20]:
for i in range(data.shape[0]):
    if data.iloc[i,-1]=='Rudolfsheim-Fnfhaus':
        data.iloc[i,-1]='Rudolfsheim-Fünfhaus'
    elif data.iloc[i,-1]=='Landstra§e':
        data.iloc[i,-1]='Landstrasse'
    elif data.iloc[i,-1]=='Whring':
        data.iloc[i,-1]='Währing'
    elif data.iloc[i,-1]=='Dbling':
        data.iloc[i,-1]='Döbling'

### Change the data type of Person capacity to integer

In [21]:
data['Person capacity'] = data['Person capacity'].astype("int")

# Store the data into a csv file which will be used in the streamlit script

In [22]:
data.to_csv("airbnb_prices_in_6_european_cities.csv")